In [1]:
import numpy as np
import pysolr

np.random.seed(42)
import spacy
import sys
import pandas as pd


In [2]:
# In case your sys.path does not contain the base repo, go there.
print(sys.path)
%cd '/Users/axelsirota/repos/ml-solr-course'

['/Users/axelsirota/repos/ml-solr-course/4-ner/lab9', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [3]:
nlp = spacy.load("en_core_web_trf")
dataset = pd.read_csv('dataset/new_york_reduced.csv')[:100][["id", "name", "description", "neighbourhood_cleansed", "property_type"]]
dataset.head()

,id,name,description,neighbourhood_cleansed,property_type
0,2595,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Midtown,Entire apartment
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Bedford-Stuyvesant,Entire guest suite
2,5121,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,Bedford-Stuyvesant,Private room in apartment
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,Sunset Park,Entire apartment
4,5178,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Midtown,Private room in apartment


In [4]:
first_description = dataset["description"].iloc[0]
first_description



'Beautiful, spacious skylit studio in the heart of Midtown, Manhattan. <br /><br />STUNNING SKYLIT STUDIO / 1 BED + SINGLE / FULL BATH / FULL KITCHEN / FIREPLACE / CENTRALLY LOCATED / WiFi + APPLE TV / SHEETS + TOWELS<br /><br /><b>The space</b><br />- Spacious (500+ft²), immaculate and nicely furnished & designed studio.<br />- Tuck yourself into the ultra comfortable bed under the skylight. Fall in love with a myriad of bright lights in the city night sky. <br />- Single-sized bed/convertible floor mattress with luxury bedding (available upon request).<br />- Gorgeous pyramid skylight with amazing diffused natural light, stunning architectural details, soaring high vaulted ceilings, exposed brick, wood burning fireplace, floor seating area with natural zafu cushions, modern style mixed with eclectic art & antique treasures, large full bath, newly renovated kitchen, air conditioning/heat, high speed WiFi Internet, and Apple TV.<br />- Centrally located in the heart of Midtown Manhatta

In [5]:
doc = nlp(first_description)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Midtown, Manhattan 50 68 LOC
1 107 108 CARDINAL
Midtown Manhattan 983 1000 LOC


In [6]:
tags = []

def update_caches(document):
    doc = nlp(document)
    inner_tags = []
    for ent in doc.ents:
        inner_tags.append(ent.text)
    tags_to_append = inner_tags if len(inner_tags) else None
    tags.append(tags_to_append)

update_caches = np.vectorize(update_caches)

In [7]:
_ = update_caches(dataset[["description"]].values)

In [8]:
dataset = pd.concat([dataset, pd.Series(tags, name="tags")], axis=1)
dataset["id"] = pd.to_numeric(dataset["id"])

In [9]:
dataset.to_csv("./4-ner/lab9/expanded_dataset.csv", index=False)